In [ ]:
import os

In [ ]:
from PIL import Image     # pillow calls image
from tqdm import tqdm       # tqdm shows processing
import copy 
import torchvision
from torchvision import transforms, models   # torchvision - pretrained model for image, video in pytorch 
from torch import optim        # optim: package for optimization algorithm 
import torch.nn as nn
from torch.optim import lr_scheduler    #lr_schedule adjusts learning rate based on number of epochs
import torch     # machine learning library
import numpy as np

In [ ]:
transform_train = transforms.Compose([     #transforming image  #combines all transforms into one
        transforms.RandomResizedCrop(224),   # extracts (224, 224) size from input image randomly. crops random location of the image 
        transforms.RandomHorizontalFlip(),  # just flipping it , part of data augmentation 
        transforms.ToTensor(),      # converts image to pytorch tensor 
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])    # within range -1,1  keep value center around 0 
    ])       # [mean], [std] input data scaling, distribution - makes data within range 3 cause RGB
transform_valid = transforms.Compose([
        transforms.Resize(256),   # resize input image to (256,256)
        transforms.CenterCrop(224),   # crops center part of the image shape (224,224)
        transforms.ToTensor(),   #for image transformation
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # always use 0.5 since we are getting PIL images
    ])

In [ ]:
from glob import glob           # to check directory 

In [ ]:
print(len(glob('../input/plant-pathology-2021-fgvc8/train_images/*')))             # count dataset 
print(len(glob('../input/plant-pathology-2021-fgvc8/test_images/*')))      

In [ ]:
glob('../input/plant-pathology-2021-fgvc8/*')        # shows what's in the directory * 

In [ ]:
import random
with open('../input/plant-pathology-2021-fgvc8/train.csv', 'r') as f:   # 'r': reading
    csv = f.readlines()[1:]    # excludes 0 cause headline 
    for _ in range(5):            #randomize data so not in order range(5): 5 times
        random.shuffle(csv)
    cnt = int(len(csv)*0.9)      # cnt = 90% of data 
    train_csv = csv[:cnt]           # from beginning to cnt 
    valid_csv = csv[cnt:]                  # from cnt: end 

In [ ]:
test = {i.split(',')[1] for i in train_csv}  # train name and label -> split-> just label ([1])
test2 = {i.split(',')[1] for i in valid_csv}
print({label:idx for idx, label in enumerate(test)})  # give index for each label 
print({label:idx for idx, label in enumerate(test2)}) 

In [ ]:
class torchvision_Dataset(torch.utils.data.Dataset): 
    def __init__(self, data_root, csv, transforms=None):     # read and save meta data: data that explains other data
        self.data = csv    #use self so you can call it later on on a different cell
        self.image_path = data_root     # root: directory where you want to save the dataset
        label = {i.split(',')[1] for i in self.data}  # split label and name of image from csv file
        self.label = {label:idx for idx, label in enumerate(label)}   # enumerate indexes each label  so label:index 
        self.transform = transforms
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):                    # calls image data
        image_name, label_name = self.data[idx].split(',')  # split so can get image_name, label_name 
        img = Image.open(os.path.join(self.image_path, image_name))
        if self.transform:
            x = self.transform(img)
        return x, self.label[label_name]

In [ ]:
train_dataset = torchvision_Dataset('../input/plant-pathology-2021-fgvc8/train_images', train_csv, transform_train)
valid_dataset = torchvision_Dataset('../input/plant-pathology-2021-fgvc8/train_images', valid_csv, transform_valid)

In [ ]:
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4) # to reduce overfitting Group into 16 number of batches and shuffle within the batch 
valid_dataloaders = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4)  #num_workers: cpu->gpu faster

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   #to use gpu for torch 
print(device)

In [ ]:
class Resnet50(nn.Module):
    def __init__(self, num_classes =12):
        super().__init__()
        resnet = torchvision.models.resnet50(pretrained=True)
        resnet.fc = nn.Linear(in_features=resnet.fc.in_features, out_features = num_classes, bias= True)
        self.base_model = resnet
    def forward(self,x):
        return self.base_model(x)

In [ ]:
model_ft = Resnet50(num_classes = 12)
model_ft.to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9) # learning rate: size of step momentum: speeds (previous step matters the most)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Train Model 

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0.0
        train_corrects = 0
        train_data_cnt = 0
        train_progress_bar = tqdm(train_dataloaders)
        
        for inputs, labels in train_progress_bar: # image, label name from get item (data augmentation applied )
            model.train()
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)   # put inputs into the model predict class -> output
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)   # calculate loss of the predicted output 
            
            loss.backward()   # backprop
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            train_corrects += torch.sum(preds == labels.data)
            train_data_cnt += inputs.size(0)
            train_progress_bar.set_description(f" Epoch[{epoch+1}/{num_epochs}] train : runing_Loss {running_loss / train_data_cnt:.5f}, train_acc {train_corrects / train_data_cnt:.5f}")
        
        scheduler.step()
        
        valid_corrects = 0
        valid_data_cnt = 0
        valid_progress_bar = tqdm(valid_dataloaders)
        for inputs, labels in valid_progress_bar:
            model.eval()
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            with torch.no_grad():
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                
            valid_corrects += torch.sum(preds == labels.data)
            valid_data_cnt += inputs.size(0)
            valid_progress_bar.set_description(f" Epoch[{epoch+1}/{num_epochs}] valid : valid_acc {valid_corrects / valid_data_cnt}")
            
        epoch_acc = running_corrects / valid_dataset.__len__() 
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_epoch = epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            print(f"best epoch : {best_epoch}")
    model.load_state_dict(best_model_wts)
    return best_model_wts

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=2)

In [ ]:
# checkpoint = {'model': Resnet50(num_classes =12),
#               'state_dict':model_ft.state_dict(),
#               'optimizer': optimizer_ft.state_dict()}
# torch.save(checkpoint, 'checkpoint.pth')

Submission

In [ ]:
from glob import glob
import csv
img_paths = glob("../input/plant-pathology-2021-fgvc8/test_images/*")
idx2label = {idx:label.strip() for idx, label in enumerate(test2)}

submission = open('submission.csv', 'w', newline='')
wr = csv.writer(submission)
wr.writerow(['image','label'])


best_model =  models.resnet50(pretrained=True)
best_model.fc = torch.nn.Linear(2048,12)
best_model.load_state_dict(model_ft)
best_model.cuda()


for img_path in img_paths:
    best_model.eval()
    img = Image.open(img_path)
    img = transform_valid(img)
    img = img.unsqueeze(0)
    with torch.no_grad():
        pred = best_model(img.cuda())
    print(pred)
    _,top_one = torch.max(pred,1)
    print(top_one)
    print(idx2label[int(top_one)])
    img_path = img_path.split('/')[-1]
    print(img_path, ',', idx2label[int(top_one)])
    
    
    wr.writerow([img_path, idx2label[int(top_one)]])
    
submission.close()